# Mental health counseling ChatGPT Clone from Scratch

## Task 1: Import the Libraries

In [1]:
# Import the libraries here
import os
from datasets import load_dataset
import json
from enum import Enum 
import random
from openai import OpenAI

## Task 2: Create Classes for the Dataset

In [2]:
#  Define the RoleType Enum here
class RoleType(Enum):
    USER = 'user',
    SYSTEM = 'system'
    ASSISTANT = 'assistant'


In [3]:
# Define the Role class here
class Role:
    def __init__(self,role_type: RoleType, content):
        self.role = role_type.value
        self.content= content
        self.value = {'role':self.role, 'content':self.content}



In [4]:
#  Define the messsage class here
class Message(object):
    def __init__(self, user_content,system_content, assistant_content):
        self.user_role =Role(role_type=RoleType.USER,content=user_content)
        self.system_role =Role(role_type=RoleType.SYSTEM,content=system_content)
        self.assistant_role =Role(role_type=RoleType.ASSISTANT,content=assistant_content)
        self.message = {'message':[self.system_role.value, self.user_role.value,self.assistant_role.value]}


## Task 3: Load, Explore, and Store the Data

In [5]:
# Load the dataset 
dataset = load_dataset(
    "Amod/mental_health_counseling_conversations",
    data_files = "combined_dataset.json",
    split = "train"
)

README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
# Create a sample Message object
context = dataset[152]["Context"]
response = dataset[152]["Response"]
system_content = "You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."

message_obj = Message(user_content=context,system_content=system_content,assistant_content=response)

print(message_obj.message)

{'message': [{'role': 'system', 'content': 'You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibi

In [7]:
#  Create the train_dataset variable
sampled_dataset = random.choices(dataset,k=100)
train_dataset=[]
for row in sampled_dataset:
    message_obj = Message(user_content=row['Context'],system_content=system_content,assistant_content=row['Response'])
    train_dataset.append(message_obj)

print(train_dataset[1])

In [8]:
# Save data in JSONl format 

def save_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for row in data:
            # Extract the dict representation
            if hasattr(row, "message"):
                row = row.message
            line = json.dumps(row)
            file.write(line + '\n')

training_data_path = '/usercode/data/train.jsonl'
save_to_jsonl(train_dataset[:-5], training_data_path)

validation_data_path = '/usercode/data/validation.jsonl'
save_to_jsonl(train_dataset[-5:], validation_data_path)

## Task 4: Fine-Tune the Model

In [10]:
# Load the training and validation files
training_data = open(training_data_path,"rb")
validation_data = open(validation_data_path,"rb")


In [11]:
#  Add OpenAI api_key
import os
api_key = os.getenv("OPENAI_API_KEY")  
client = OpenAI(api_key=api_key)

In [13]:
# Upload the training and validation files
training_response = client.files.create(file=training_data, purpose="fine-tune")
training_file_id = training_response.id

validation_response = client.files.create(file=validation_data, purpose="fine-tune")
validation_file_id = validation_response.id

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-J6W8wnDZg1bpefCco7a2U8
Validation file id: file-KEGctcw4HVhmTzRDdUgL2i


In [14]:
# Create a fine-tuning job

response = client.fine_tuning.jobs.create(
    training_file = training_file_id,
    model = "gpt-3.5-turbo",
    suffix = "my-test-model",
    validation_file = validation_file_id
)

job_id = response.id 
print(response)

FineTuningJob(id='ftjob-m4prvOzqAMLHGyInCQ2G6eBX', created_at=1757108301, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-CYfjXQZ9mV4tQpn1I451OOD8', result_files=[], seed=2099961985, status='validating_files', trained_tokens=None, training_file='file-J6W8wnDZg1bpefCco7a2U8', validation_file='file-KEGctcw4HVhmTzRDdUgL2i', estimated_finish=None, integrations=[], user_provided_suffix='my-test-model', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})


In [16]:
# Retrieve the job status
job_status= client.fine_tuning.jobs.retrieve(job_id)
print(job_status)

FineTuningJob(id='ftjob-m4prvOzqAMLHGyInCQ2G6eBX', created_at=1757108301, error=Error(code='invalid_file_format', message='The job failed due to a file format error in the training file. Invalid file format. Example 1 is missing key "messages".', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-CYfjXQZ9mV4tQpn1I451OOD8', result_files=[], seed=2099961985, status='failed', trained_tokens=None, training_file='file-J6W8wnDZg1bpefCco7a2U8', validation_file='file-KEGctcw4HVhmTzRDdUgL2i', estimated_finish=None, integrations=[], user_provided_suffix='my-test-model', metadata=None, usage_metrics=None, shared_with_openai=False, eval_id=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 'auto', 'batch_size': 'auto', 'learning_rate_multiplier': 'auto'}}})


## Task 5: Test the Fine-Tuned Model

In [17]:
# Create and store message dictionaries
system_message = """You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user's emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It's important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."""

messages = []
messages.append({"role": "system", "content": system_message})
user_message = "Every winter I find myself getting sad because of the weather. How can I fight this?"
messages.append({"role": "user", "content": user_message})

In [18]:
# Test the fine-tuned chat completion model
completion = client.chat.completions.create(
    model = response.model,
    messages = messages
)

print(completion.choices[0].message)

ChatCompletionMessage(content="I'm sorry to hear that you struggle with feeling sad during winter. It's not uncommon to experience seasonal changes affecting our mood. One way to combat this is by trying to bring more light into your daily routine. Getting outdoors during daylight hours, even for a short walk, can help boost your mood. Additionally, light therapy using a lightbox can mimic natural sunlight and be helpful for some people.\n\nIt's also important to stay connected with others, even if it's just through phone calls or virtual meetings. Social support can make a big difference in how we feel. Engaging in activities you enjoy, staying physically active, and practicing self-care like getting enough sleep and eating well can also help improve your mood.\n\nIf you find that these strategies aren't bringing relief or if your feelings of sadness become overwhelming, it might be beneficial to speak with a therapist or counselor who can provide support and guidance tailored to your

In [19]:
# Get and compare the output of the gpt-3.5-turbo chat completion model

completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = messages
)

print(completion.choices[0].message)


ChatCompletionMessage(content="It sounds like you may be experiencing seasonal affective disorder (SAD), which can affect many people during the winter months. It's great that you're looking for ways to fight this and take care of yourself. Here are some tips that might help:\n\n1. **Light Therapy:** Consider using a light therapy box, which mimics natural sunlight, to help improve your mood.\n\n2. **Spending Time Outdoors:** Try to get outside during daylight hours, even if it's cloudy. Fresh air and natural light can have a positive impact on your mood.\n\n3. **Regular Exercise:** Physical activity can help boost your mood and energy levels. Even a short walk or some gentle stretching can make a difference.\n\n4. **Healthy Diet:** Eating well-balanced meals and staying hydrated can positively influence your mood and overall well-being.\n\n5. **Social Support:** Make an effort to stay connected with friends and loved ones. Social interaction can help combat feelings of isolation and l